In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Telangana_bus_details.csv', index=False)

driver.quit()

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 64089,Rajdhani (AC Semi Sleeper 2+2),18:00,03h 20m,21:20,3.3,INR 389,28 Seats available
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8822,Super Luxury (Non AC Seater 2+2 Push Back),18:15,03h 45m,22:00,3.4,INR 320,24 Seats available
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 4172,Rajdhani (AC Semi Sleeper 2+2),18:30,04h 00m,22:30,3.3,INR 389,35 Seats available
3,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 64022,Rajdhani (AC Semi Sleeper 2+2),19:00,03h 30m,22:30,3.1,INR 389,30 Seats available
4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8677,Super Luxury (Non AC Seater 2+2 Push Back),19:30,04h 10m,23:40,3.2,INR 320,31 Seats available
...,...,...,...,...,...,...,...,...,...,...
869,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),23:05,03h 40m,02:45,2.7,INR 600,26 Seats available
870,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),23:10,03h 50m,03:00,2.8,INR 600,23 Seats available
871,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),23:15,05h 35m,04:50,2.7,404,27 Seats available
872,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),23:25,03h 55m,03:20,2.7,INR 600,20 Seats available
